In [13]:
import os
import requests
import json
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [3]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")

# check the key
if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [5]:
# Some websites need to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class ScrapeWebsite:
    """
    A utility class to represent a Website that we have scraped, now with links
    """
    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [14]:
class MakeCompanyBrochure:
    def __init__(self, company_name, url):
        self.main_website = ScrapeWebsite(url)
        self.company_name = company_name

    def get_links(self):
        system_prompt = "You are provided with a list of links found on a webpage. \
            You are able to decide which of the links would be most relevant to include in a brochure about the company, \
            such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
        system_prompt += "You should respond in JSON as in this example:"
        system_prompt += """
        {
            "links": [
                {"type": "about page", "url": "https://full.url/goes/here/about"},
                {"type": "careers page": "url": "https://another.full.url/careers"}
            ]
        }
        """

        user_prompt = f"Here is the list of links on the website of {self.main_website.url} - "
        user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
            Do not include Terms of Service, Privacy, email links.\n"
        user_prompt += "Links (some might be relative links):\n"
        user_prompt += "\n".join(self.main_website.links)

        response = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
          ],
            response_format={"type": "json_object"}
        )
        result = response.choices[0].message.content
        return json.loads(result)

    def get_all_details(self):
        result = "Landing page:\n"
        result += self.main_website.get_contents()
        links = self.get_links()
        print("Found links:", links)
        
        for link in links["links"]:
            result += f"\n\n{link['type']}\n"
            result += ScrapeWebsite(link["url"]).get_contents()
        return result

    def create_brochure(self):
        system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
            and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
            Include details of company culture, customers and careers/jobs if you have the information."

        user_prompt = f"You are looking at a company called: {self.company_name}\n"
        user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
        user_prompt += self.get_all_details()
        user_prompt = user_prompt[:5_000]
        
        stream = openai.chat.completions.create(
            model=MODEL,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
              ],
            stream=True
        )
        
        response = ""
        display_handle = display(Markdown(""), display_id=True)
        for chunk in stream:
            response += chunk.choices[0].delta.content or ''
            response = response.replace("```","").replace("markdown", "")
            update_display(Markdown(response), display_id=display_handle.display_id)

In [17]:
brochure = MakeCompanyBrochure("SafetyCulture", "https://safetyculture.com/")

In [18]:
brochure.create_brochure()

Found links: {'links': [{'type': 'about page', 'url': 'https://safetyculture.com/about/'}, {'type': 'careers page', 'url': 'https://safetyculture.com/careers/'}, {'type': 'company page', 'url': 'https://safetyculture.com/newsroom/'}, {'type': 'contact page', 'url': 'https://safetyculture.com/contact-us/'}, {'type': 'team page', 'url': 'https://safetyculture.com/meet-the-team/'}]}


# SafetyCulture Company Brochure

## Welcome to SafetyCulture

At SafetyCulture, we empower teams to improve workplace operations, enhancing safety, efficiency, and overall operational excellence. Our innovative platform provides everything your teams need to work safely, meet higher standards, and improve every day.

### Our Product Offering

**Workplace Operations Platform**  
Our all-in-one digital solution includes:

- **Inspections**: Streamline audits and checklists, digitizing processes for instant data capture and reporting.
- **Assets**: Gain full visibility of your assets with a digital register of their history and future activities.
- **Training**: Create on-the-job courses quickly to onboard and upskill team members effectively.
- **Sensors & IoT**: Utilize technology for real-time monitoring and management.
- **Integrations**: Seamlessly connect with other tools and systems.
- **Lone Worker**: Enhance safety for remote workers with real-time protections.
- **Marketplace**: Your one-stop shop for workplace safety equipment.
- **Insurance**: Introducing SafetyCulture Care, a unique blend of risk management, technology, and insurance.
- **HSEQ Documentation**: Access comprehensive and trusted safety, health, quality, and environmental documents with SafetyDocs.

### Who We Serve

Our solutions cater to a range of industries including:

- **Construction**
- **Manufacturing**
- **Facilities Management**
- **Hospitality**
- **Retail**
- **Transport & Logistics**
- **Mining**
- **Healthcare & Pharmaceuticals**

### Business Needs Addressed

We support organizations of all sizes with:

- **Health & Safety Management (EHS)**
- **Risk Management and Compliance (GRC)**
- **Operational Excellence**
- **Environment & Sustainability (ESG)**

### Company Culture

At SafetyCulture, we believe in promoting a safety culture that empowers every individual. We foster an environment of collaboration, transparency, and continuous improvement, encouraging our teams to take an active role in improving quality, safety, and compliance. Our partnerships with notable organizations like the Australian Open exemplify our commitment to delivering exceptional experiences while prioritizing workforce engagement and safety.

### Join Our Team!

We are constantly looking for passionate and driven individuals to join our growing team. If you are interested in shaping the future of workplace safety and operational efficiency, we welcome your application. 

Check our [Careers page](#link) for current job openings and become a part of our mission to improve workplace operations globally!

### Ready to Transform Your Workplace?

Explore how SafetyCulture can make a difference in your operations. **[Book a demo today!](#link)**

---

For additional information, insights, and resources, visit our [website](#link) or [contact our support team](#link), who will be happy to assist you.

---

**SafetyCulture: Improve Workplace Operations, Every Day!**